In [1]:
import numpy as np

In [ ]:
import wikipedia
print wikipedia.summary("アルコ&ピースのオールナイトニッポンシリーズ")

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("contents.csv",index_col=0)

In [ ]:
for i in data.index:
    print wikipedia.summary(i)

In [ ]:
import lxml.html 
import urllib3
import json
import requests

urllib3.disable_warnings()
http = urllib3.PoolManager()
# r = http.request('GET', 'http://ja.wikipedia.org/w/api.php?format=xml&action=query&prop=revisions&titles=朝井リョウ%26加藤千恵のオールナイトニッポン0(ZERO)&rvprop=content')
r = http.request('GET', 'http://ja.wikipedia.org/w/api.php?format=json&action=query&prop=revisions&titles=朝井リョウ%26加藤千恵のオールナイトニッポン0(ZERO)&rvprop=content')
print r.data

#print data
# html = lxml.html.fromstring(data)
# title = html.xpath('query/pages/page/@title')[0];
# print(html)
# print(r.data)
# print(title)


In [ ]:
import json
import requests
url =  'http://ja.wikipedia.org/w/api.php?format=json&action=query&prop=revisions&titles=朝井リョウ%26加藤千恵のオールナイトニッポン0(ZERO)&rvprop=content'
# url =  'http://ja.wikipedia.org/w/api.php?format=json&action=query&prop=revisions&titles=ウレロ☆シリーズ&rvprop=content'
# url =  'http://en.wikipedia.org/w/api.php?format=json&action=query&prop=revisions&titles==Emma_Watson&rvprop=content'
# url =' http://ja.wikipedia.org/w/api.php?format=json&action=query&prop=revisions&titles=Main%20Page&rvprop=content'
# url = 'https://en.wikipedia.org/w/api.php?action=query&titles=Main%20Page&prop=revisions&rvprop=content&format=json&rvprop=content'
# r = requests.get(url)
# data = r.json()
# print data # print json.dumps(data, indent=4)
resp = requests.get(url).json()
# print resp.keys(), resp.values()
page_one = next(iter(resp['query']['pages'].values()))
#print page_one
# print len(resp['query']['pages'])
# print page_one.keys()
# print page_one['title']
revisions = page_one.get('revisions', [])
#print revisions
# html = next(iter(revisions[0].values()))

print revisions[0]['*']
# print revisions[0][u'contentmodel']
# print revisions[0][u'contentformat']

# now parse the html 


# r = http.request('GET', url)
# print r.data
# html = lxml.html.fromstring(r.data.decode('utf-8'))
# title = html.xpath('query/pages/page/@title')[0];



In [ ]:
# the way to get infobox from wikipedia with pandas
import pandas
# fetched_dataframes = pandas.io.html.read_html('https://ja.wikipedia.org/wiki/%E6%9C%9D%E4%BA%95%E3%83%AA%E3%83%A7%E3%82%A6%26%E5%8A%A0%E8%97%A4%E5%8D%83%E6%81%B5%E3%81%AE%E3%82%AA%E3%83%BC%E3%83%AB%E3%83%8A%E3%82%A4%E3%83%88%E3%83%8B%E3%83%83%E3%83%9D%E3%83%B30(ZERO)')
# url = 'https://ja.wikipedia.org/wiki/朝井リョウ%26加藤千恵のオールナイトニッポン0(ZERO)'
url ='https://ja.wikipedia.org/wiki/%E3%82%A6%E3%83%AC%E3%83%AD%E2%98%86%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA'
infobox = pandas.read_html(url, match=u'ジャンル')
for i in infobox:
    print infobox[0].describe

In [ ]:
#usage of wikipedia package
import wikipedia
wikipedia.set_lang('ja')

# print wikipedia.summary('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
# p = wikipedia.page('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
p = wikipedia.page('ウレロ☆シリーズ')

# print p.content
# print p.categories
# print dir(p)
print 'カテゴリー'
for v in p.categories:
    print ' ', v

#print p.content
#print p.html()

In [ ]:
import re
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser, object):
    def __init__(self):
        super(MyHTMLParser, self).__init__()
        self._find_headline = False
        self._find_staff = False
#         self._uli_count= 0
        self._li_count= 0
        self._dd_count= 0
        self._bracket_count = 0
        self._find_ref = False
#         self._find_table = False
#         self._find_colon = False
        
        self._index = ''
        self._data_temp = ''
        self._value = []
        self._value_set = set()
        self._staff_dict = dict()
        
    def handle_starttag(self, tag, attrs):
        if self._find_staff:
#             print("Start tag:", tag)
            if tag == u'a':
                self._link = True
            if tag == u'li':
                self._li_count +=1
                self._data_temp = ''
            if tag == u'dd':
                self._dd_count += 1
#             if tag ==
       
        for attr in attrs:
#             if self._find_staff:
#                 print("     attr:", attr)
            if 'mw-headline' in attr:
                self._find_headline = True
                self._find_staff = False
            if 'reference' in attr:
                self._find_ref = True

            
    def handle_endtag(self, tag):
        if self._find_staff: 
#             print("End tag  :", tag)
#             if tag == u'uli':
#                 self._uli_count -= 1
            if tag == u'dd':
                self._dd_count -= 1
                
                datas = re.split(u'[：-]', self._data_temp)
                self._index = datas[0]
                self._value.extend(re.split(u'[,、・/／→]', datas[1])) #separate multiple staffs
                for i, v in enumerate(self._value): #remove () and ※
                    self._value[i]  = re.split(u'[（(※]', v)[0]
                self._staff_dict[self._index] = self._value                

                self._value = []

            if tag == u'li':
                self._li_count -= 1
#                 self._find_colon = False
                datas = re.split(u'[：-]', self._data_temp)
                self._index = datas[0]

                datae = datas[1].encode('utf-8')
                datae = re.sub('(（|\(|\（).+?(）|\）|\))','', datae) #remove brackets

                #exception
                exception = ['A・T・C事務所']
                for e in exception:
                    if e in datae:
                        self._value.append(e)
                        datae = datae.replace(e,'')
                
                self._value.extend(re.split(u'[,、・/／→]', datae.decode('utf-8'))) #separate multiple staffs                
                for i, v in enumerate(self._value): #remove () and ※
                    self._value[i]  = re.split(u'[※]', v)[0]
#                     print '    ',  self._value[i]
               
                if self._index in self._staff_dict:
                    self._staff_dict[self._index].extend(self._value)
                else:
                     self._staff_dict[self._index] = self._value
                
                self._value = []

        if self._find_headline and tag==u'span':
            self._find_headline = False

    def handle_data(self, data):
#         if self._find_staff: 
#             print data
#             print self._li_count, self._dd_count
        if self._find_staff and self._li_count>0 or self._dd_count>0:
                 if not self._find_ref :
                        self._data_temp += data
#                 print self._data_temp
#             if not self._find_colon:
#                 self._find_colon = True
#                 datas = re.split(u'[：-]', data)
#                 self._index = datas[0]
#                 self._value.extend(re.split(u'[,・/,／]', datas[1]))
#             else:
#                 self._value.extend(re.split(u'[,・/,／]', data))
        if self._find_ref :
            self._find_ref = False
            
        if self._find_headline:
            if data == u'スタッフ':
                self._find_staff = True
                self._li_count = 0


parser = MyHTMLParser()
parser.feed(p.html())

for k, v in parser._staff_dict.items():
    print k, ':', 
    for staff in set(v):
        print staff,
    print ''

In [ ]:
infobox = pandas.read_html(p.html(), match=u'ジャンル')
# for i in infobox:
#     print infobox[0].describe
    
ib = infobox[0]
ib = ib.set_index(ib[0])
ib = ib.drop(u'テンプレートを表示')
ib = ib.drop(0, axis=1)
# print ib.at[u'出演者',1]

ib

# print ib
# ib = ib.drop('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)  ', axis=1)
# ib.drop(u'公式サイト', axis=0)
# print ib
ib.loc['test'] = [ ['ddd', 'tst', 'ttt']]
ib
# print ib[0]
# ib[ib[0]==u'公式サイト']
# print ib.drop(ib[ib[0]==u'テンプレートを表示'])
# ib.query('0=='公式サイト')
# ib.drop(u'公式サイト', index_col=0)

In [ ]:
import wikipedia
p = wikipedia.page('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
# p = wikipedia.page('ウレロ☆シリーズ')

In [ ]:
import re
from pyquery import PyQuery as pq
query = pq(p.html(), parser='html')
ib = query.find('.infobox')

# parse infobox table from html
staff_dict = dict()
for tr in ib.find('tr'):
    index = pq(tr).find('th').text()
    value = pq(tr).find('td').text()
    value = re.sub('(（|\(|\（).+?(）|\）|\))','', value.encode('utf-8')) #remove brackets
    if index is not '' and v is not '':    
        value = value.split('\n')
        if i in  staff_dict:
            staff_dict[index].extend(value)
        else:
             staff_dict[index] = value

# output
for k, v in staff_dict.items():
    print k, ':', 
    for staff in set(v):
        print staff,', ',
    print ''

In [ ]:
import lxml.html as xh

find_staff = False
staff_header = ''
staff_end = ''

heads = query.find('.mw-headline')
for h in heads:
    if find_staff:
        staff_end = pq(h)
        break
    if h.text == u'スタッフ':       
        find_staff = True
#         print type(h)
#         print '&#12473;&#12479;&#12483;&#12501;'.encode('utf-8')
#         print [h.text.encode('utf-8')]
        print h, h.text
        staff_header = pq(h)
        print [staff_header.text()|]
        staff_header = xh.tostring(h)
        print staff_header.decode('unicode-escape')

# print query.html().find(u'id="スタッフ"')
# print [query.html()]

# print staff_header
# print staff_header.find(u'スタッフ')

# print type(html.encode('utf-8')), type(staff_header)

# print staff_end.text()
# print type(html.encode('utf-8'))
# print type(staff_header.html())

# uls = query.find('ul')
# for ul in uls:
#     lis = ul.find('li')
#     for li in lis:
#         print li.tex

# query('li').outerHtml()


In [ ]:
import wikipedia
wikipedia.set_lang('ja')
p = wikipedia.page('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
# p = wikipedia.page('ウレロ☆シリーズ')

In [ ]:
import bs4.element as bs4elem
from bs4 import BeautifulSoup

bsObj = BeautifulSoup(p.html(), "html.parser")

htmls = []
headers = bsObj.find_all(class_='mw-headline')

find_staff = False
staff_start = ''
staff_end = ''
for h in headers:
#         if find_staff:
#             staff_end_class = h
#             break
        if h.text == u'スタッフ':
            staff_start = h
#             break
            find_staff = True
    
last_text = ''
def find_list(elt, parent=''):
    global last_text
    for e in elt.find_all('ul'):
        for s in e.parent.strings:
            parent += s.strip('\n') #repr(s).decode('unicode-escape')
            break            
        find_list(e, parent)
        parent = ''
        e.parent.decompose()
    for e in elt.find_all(['dd', 'li']):
        if e.name == 'dd':
            print last_text + e.text
            last_text = ''
        else:
            last_text = parent+e.text
            print parent+e.text        
#     

finish = False
for elt in staff_start.parent.nextSiblingGenerator():
    for e in elt:
        if isinstance(e, bs4elem.Tag) and e.get('class') == [u'mw-headline']:
            finish = True
            break
    if finish:
        break

    if hasattr(elt, "text"):
        find_list(elt)
        


In [ ]:
from bs4 import BeautifulSoup

bsObj = BeautifulSoup(p.html(), "html.parser")

table = bsObj.findAll("table",{"class":"infobox"})[0]
rows = table.findAll("tr")

result = dict()

for row in rows:
    index = row.find('th')
    if index is not None:
        index = index.get_text()
        value = row.find('td')
        if value is not None:
            value = BeautifulSoup(str(value).replace('<br/>', '\n'), "html.parser")
            value = value.get_text().split('\n')
            for v in value:
                print v,',      ',
            print ''

            result[index] = value

# output
# for k, v in result.items():
#     print k, ':', 
#     for staff in set(v):
#         print staff,', ',
#     print ''

#             for i, elm in enumerate(value.childGenerator()):
#                 print(i, ":", elm)
#                 print elm.get_text()

        
#         brs = value.findAll('br')
#         print brs
#         value = value.get_text()
#         if value[:1] == '\n':
#             value = value[1:]
        
#     if index is not None and value is not None:
#         print index + u': ' + value
#     else:
#         print index, value



#     for cell in row.findAll(['td', 'th']):
#         csvRow.append(cell.get_text())
        
#     for v in csvRow:
#         print v,  
#     print ''

In [ ]:
'\n'.decode('utf-8')

In [ ]:
'h3' >= 'h4'

In [ ]:
import re
if re.match('h[1-6]', 'h1'):
    print 'test'